In [1]:
import pandas as pd
import scipy.stats as sps
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
import pandas_datareader as pdd
from datetime import datetime
from meteostat import Stations, Daily
import matplotlib.collections as mcoll
import matplotlib.path as mpath
import matplotlib.cm as cm


Bad key "mathtext.fallback" on line 156 in
C:\Users\maxim\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\classic.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key "axes.titley" on line 177 in
C:\Users\maxim\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\classic.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.0/matplotlibrc.template
or from the matplotlib source distribution


AttributeError: module 'matplotlib.transforms' has no attribute '_make_str_method'

In [ ]:
plt.rcParams['font.size'] = 18
hfont = {'fontname':'DejaVu Serif'}

In [ ]:
dr = pd.read_csv('results_group_h.csv', sep = ',', usecols = [0, 1, 2, 3], index_col = 'Trajectory')

In [ ]:
s = len(dr.groupby(dr.index))
print(s)

df = dr.groupby(dr.index).filter(lambda x: len(x) >= 50)
df = df.assign(Δx = df.groupby(df.index)['x'].diff().values)
df = df.assign(Δy = df.groupby(df.index)['y'].diff().values)
df = df.dropna()
df

In [ ]:
kc = 4 #kurtosis cutoff

dfk = df.groupby(df.index).filter(lambda x: (x['Δx'].kurtosis() + 3) <= kc and (x['Δy'].kurtosis() + 3) <= kc)
print(dfk)
e = len(dfk.groupby(dfk.index))
print('Number of used trajectories: ' + str(e))

Δs = s - e
print('Difference in number of trajectories: ' + str(Δs))

In [ ]:
max_traj = dfk.groupby(dfk.index).count()['Frame'].max()
df_max = dfk.groupby(dfk.index).filter(lambda x: len(x) == max_traj)

def colorline(x, y, cmap = plt.get_cmap('jet'), linewidth = 3, alpha = 1.0):

    z = np.linspace(0.0, 1.0, len(x))
    z = np.asarray(z)

    segments = make_segments(x, y)
    lc = mcoll.LineCollection(segments, array = z, cmap = cmap, linewidth = linewidth, alpha = alpha)

    ax = plt.gca()
    ax.add_collection(lc)

    return lc


def make_segments(x, y):
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    return segments

x = df_max['x'].values
y = df_max['y'].values

fig, ax = plt.subplots()

path = mpath.Path(np.column_stack([x, y]))
verts = path.interpolated(steps=3).vertices
x, y = verts[:, 0], verts[:, 1]
colorline(x, y, cmap = plt.get_cmap('jet'), linewidth = 1.5)
plt.grid(True) ; plt.xlabel('x', **hfont) ; plt.ylabel('y', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
cbar = plt.colorbar(cm.ScalarMappable(norm = plt.Normalize(0.0, len(x)), cmap = plt.get_cmap('jet')))
cbar.set_label('Time t',size = 18, family = 'DejaVu Serif')
for l in cbar.ax.yaxis.get_ticklabels():
    l.set_family('DejaVu Serif')
    l.set_size(14)
plt.axis([403, 470, 99, 152])
plt.show()

df_max.plot(x = 'Frame', y = 'Δx', legend = False)
plt.grid(True) ; plt.xlabel('Time t', **hfont) ; plt.ylabel('Δx', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.autoscale(axis = 'x', tight = True)
plt.show()

df_max.plot(x = 'Frame', y = 'Δy', legend = False)
plt.grid(True) ; plt.xlabel('Time t', **hfont) ; plt.ylabel('Δy', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.autoscale(axis = 'x', tight = True)
plt.show()

In [ ]:
mean_x = dfk.groupby(dfk.index)['Δx'].mean().mean()
mean_y = dfk.groupby(dfk.index)['Δy'].mean().mean()
var_x = dfk.groupby(dfk.index)['Δx'].var().var()
var_y = dfk.groupby(dfk.index)['Δy'].var().var()
skew_x = dfk.groupby(dfk.index)['Δx'].skew().skew()
skew_y = dfk.groupby(dfk.index)['Δy'].skew().skew()
kurt_x = dfk.groupby(dfk.index)['Δx'].apply(lambda x: sps.kurtosis(x, fisher = False)).apply(lambda x: sps.kurtosis(x, fisher = False))
kurt_y = dfk.groupby(dfk.index)['Δy'].apply(lambda x: sps.kurtosis(x, fisher = False)).apply(lambda x: sps.kurtosis(x, fisher = False))

In [ ]:
dfk = dfk.assign(Δx_n = dfk.groupby(dfk.index)['Δx'].apply(lambda x: (x - np.mean(x)) / np.std(x)))
dfk = dfk.assign(Δy_n = dfk.groupby(dfk.index)['Δy'].apply(lambda x: (x - np.mean(x)) / np.std(x)))
dfk

In [ ]:
cmean_x = dfk['Δx_n'].mean()
cmean_y = dfk['Δy_n'].mean()
cvar_x = dfk['Δx_n'].var()
cvar_y = dfk['Δy_n'].var()
cskew_x = dfk['Δx_n'].skew()
cskew_y = dfk['Δy_n'].skew()
ckurt_x = dfk['Δx_n'].kurtosis() + 3
ckurt_y = dfk['Δy_n'].kurtosis() + 3
                          
print(cmean_x, cmean_y, cvar_x, cvar_y, cskew_x, cskew_y, ckurt_x, ckurt_y)

In [ ]:
dfk.hist(column = 'Δx_n', density = True, bins = 100)
plt.plot(np.linspace(-4, 4, 1000), sps.norm.pdf(np.linspace(-4, 4, 1000)), linewidth = 3)
plt.xlabel('Standardized Δx', **hfont) ; plt.ylabel('Relative frequency', **hfont)
plt.title('')
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.autoscale(axis = 'x', tight = True)
plt.show()

dfk.hist(column = 'Δx_n', density = True, log = True, bins = 100)
plt.plot(np.linspace(-4, 4, 1000), sps.norm.pdf(np.linspace(-4, 4, 1000)), linewidth = 3)
plt.xlabel('Standardized Δx', **hfont) ; plt.ylabel('Relative frequency', **hfont)
plt.semilogy()
plt.title('')
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.autoscale(axis = 'x', tight = True)
plt.show()

dfk.hist(column = 'Δy_n', density = True, log = True, bins = 100)
plt.plot(np.linspace(-4, 4, 1000), sps.norm.pdf(np.linspace(-4, 4, 1000)), linewidth = 3)
plt.xlabel('Standardized Δy', **hfont) ; plt.ylabel('Relative frequency', **hfont)
plt.semilogy()
plt.title('')
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.autoscale(axis = 'x', tight = True)
plt.show()

dfk.hist(column = 'Δy_n', density = True, bins = 100)
plt.plot(np.linspace(-4, 4, 1000), sps.norm.pdf(np.linspace(-4, 4, 1000)), linewidth = 3)
plt.xlabel('Standardized Δy', **hfont) ; plt.ylabel('Relative frequency', **hfont)
plt.title('')
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.autoscale(axis = 'x', tight = True)
plt.show()

In [ ]:
ax = dfk.plot(kind = 'scatter', x = 'Δx', y = 'Δy', s = 1.5)
plt.xlabel('Δx', **hfont) ; plt.ylabel('Δy', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
ax.set_aspect('equal')

In [ ]:
def qrank(series):
    return (series.argsort().argsort() - 0.5) / len(series)

heatmap, xedges, yedges = np.histogram2d(dfk['Δx'], dfk['Δy'], bins=(10, 10))
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.ylabel('Δy', **hfont)
plt.xlabel('Δx', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.imshow(heatmap, extent = extent)
cbar = plt.colorbar()
cbar.set_label('Frequency',size = 18, family = 'DejaVu Serif')
for l in cbar.ax.yaxis.get_ticklabels():
    l.set_family('DejaVu Serif')
    l.set_size(14)
plt.show()

x_r = qrank(dfk['Δx'])
y_r = qrank(dfk['Δy'])

plt.scatter(x_r, y_r, s = 0.75)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xlabel('Ranked Δx', **hfont) ; plt.ylabel('Ranked Δy', **hfont)
plt.autoscale(tight = True)
plt.show()

heatmap, xedges, yedges = np.histogram2d(x_r, y_r, bins=(24, 24))
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.ylabel('Ranked Δy', **hfont)
plt.xlabel('Ranked Δx', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.imshow(heatmap, extent = extent)
cbar = plt.colorbar()
cbar.set_label('Frequency', size = 18, family = 'DejaVu Serif')
for l in cbar.ax.yaxis.get_ticklabels():
    l.set_family('DejaVu Serif')
    l.set_size(14)
plt.show()

In [ ]:
x = pd.plotting.autocorrelation_plot(df_max['Δx']) 
x.plot
plt.axis([0, 250, -0.2, 0.2])
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xlabel('Lag', **hfont) ; plt.ylabel('Autocorrelation Δx', **hfont)
plt.show()

y = pd.plotting.autocorrelation_plot(df_max['Δy']) 
plt.axis([0, 250, -0.2, 0.2])
y.plot
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xlabel('Lag', **hfont) ; plt.ylabel('Autocorrelation Δy', **hfont)
plt.show()

x2 = pd.plotting.autocorrelation_plot(df_max['Δx'].apply(lambda x: x ** 2)) 
x2.plot
plt.axis([0, 250, -0.2, 0.2])
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xlabel('Lag', **hfont) ; plt.ylabel('Autocorrelation Δx²', **hfont)
plt.show()

y2 = pd.plotting.autocorrelation_plot(df_max['Δy'].apply(lambda x: x ** 2)) 
y2.plot
plt.axis([0, 250, -0.2, 0.2])
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xlabel('Lag', **hfont) ; plt.ylabel('Autocorrelation Δy²', **hfont)
plt.show()

In [ ]:
x1 = df_max['Δx'][1:]
x2 = df_max['Δx'][:-1]

plt.scatter(x1, x2, s = 9)
plt.grid(True)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xlabel('Δx(t + 1)', **hfont) ; plt.ylabel('Δx(t)', **hfont)
plt.show();

y1 = df_max['Δy'][1:]
y2 = df_max['Δy'][:-1]

plt.scatter(y1, y2, s = 9)
plt.grid(True)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xlabel('Δy(t + 1)', **hfont) ; plt.ylabel('Δy(t)', **hfont)
plt.show()

In [ ]:
y1_r = qrank(y1)
y2_r = qrank(y2)

heatmap, xedges, yedges = np.histogram2d(y1_r, y2_r, bins=(16, 16))
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.ylabel('Ranked Δy(t)', **hfont)
plt.xlabel('Ranked Δy(t + 1)', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.imshow(heatmap, extent=extent)
cbar = plt.colorbar()
cbar.set_label('Frequency', size = 18, family = 'DejaVu Serif')
for l in cbar.ax.yaxis.get_ticklabels():
    l.set_family('DejaVu Serif')
    l.set_size(14)
plt.show()

x1_r = qrank(x1)
x2_r = qrank(x2)

heatmap, xedges, yedges = np.histogram2d(x1_r, x2_r, bins=(16, 16))
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.ylabel('Ranked Δx(t)', **hfont)
plt.xlabel('Ranked Δx(t + 1)', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.imshow(heatmap, extent=extent)
cbar = plt.colorbar()
cbar.set_label('Frequency', size = 18, family = 'DejaVu Serif')
for l in cbar.ax.yaxis.get_ticklabels():
    l.set_family('DejaVu Serif')
    l.set_size(14)
plt.show()

In [ ]:
shares = ['PFE', 'GME', 'AMZN', 'AZN', 'DDAIF', 'TSLA', 'TYEKF', 'VWDRY', 'DGV2.F', 'RWEOY']   
#Pfizer, GameStop, Amazon, Astrazeneca, Mercedes, Tesla, Thyssenkrupp, Vestas Wind Systems, ABACUS MNG (Kohle), RWE

yf.pdr_override()
raw_data = pdd.get_data_yahoo(shares, start = '2011-02-11', end = '2021-02-11')
data = raw_data['Adj Close']
data

In [ ]:
data['RWEOY'].plot()
plt.grid(True)
plt.ylabel('Adjusted closing price', **hfont)
plt.xlabel('Date', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.autoscale(axis = 'x', tight = True);

In [ ]:
R = data.diff().div(data).dropna()
plt.ylabel('Returns', **hfont)
plt.xlabel('Date', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14);
R['RWEOY'].plot()
plt.grid(True)
plt.autoscale(axis = 'x', tight = True)
plt.show()
R

In [ ]:
for share in shares:
    print(share + ' Mean: ' + str(R[share].mean()))
    print(share + ' Var: ' + str(R[share].var()))
    print(share + ' Skew: ' + str(R[share].skew()))
    print(share + ' Kurt: ' + str(R[share].kurtosis() + 3))

R_k = R.sub(R.mean()).divide(R.std())
R_k

In [ ]:
for i, share in enumerate(shares):
    R_k.hist(column = share, density = True, bins = 100)
    plt.plot(np.linspace(-4, 4, 1000), sps.norm.pdf(np.linspace(-4, 4, 1000)), linewidth = 3)
    if i in [0, 2, 3, 4, 5, 7, 9]:
        plt.axis([-4, 4, 0, 0.7])
    else:
        plt.axis([-4, 4, 0, 1.2])
    plt.title(share, **hfont)
    plt.ylabel('Relative frequency', **hfont)
    plt.xlabel('Standardized Returns', **hfont)
    plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
    plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
    plt.show()

    R_k.hist(column = share, density = True, bins = 100, log = True)
    plt.semilogy()
    plt.plot(np.linspace(-4, 4, 1000), sps.norm.pdf(np.linspace(-4, 4, 1000)), linewidth = 3)
    plt.axis([-4, 4, 1e-4, 1.2])
    plt.title(share, **hfont)
    plt.ylabel('Relative frequency', **hfont)
    plt.xlabel('Standardized Returns', **hfont)
    plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
    plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
    plt.show()

In [ ]:
plt.scatter(R_k['PFE'], R_k['AZN'])
plt.ylabel('AstraZeneca Returns', **hfont)
plt.xlabel('Pfizer Returns', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.show()

In [ ]:
plt.scatter(R_k['TSLA'], R_k['GME'])
plt.ylabel('GameStop Returns', **hfont)
plt.xlabel('Tesla Returns', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.show()

In [ ]:
plt.scatter(R_k['DDAIF'], R_k['TSLA'])
plt.ylabel('Tesla Returns', **hfont)
plt.xlabel('Daimler Returns', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.show()

In [ ]:
plt.scatter(R_k['VWDRY'], R_k['DGV2.F'])
plt.show()

In [ ]:
pfe = qrank(R_k['PFE'])
azn = qrank(R_k['AZN'])

plt.scatter(pfe, azn, s = 4)
plt.xlabel('Pfizer ranked Returns', **hfont)
plt.ylabel('AstraZeneca ranked Returns', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.autoscale(tight = True)
plt.show()

heatmap, xedges, yedges = np.histogram2d(pfe, azn, bins=(16, 16))

plt.clf()
plt.xlabel('Pfizer ranked Returns', **hfont)
plt.ylabel('AstraZeneca ranked Returns', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.imshow(heatmap, origin = 'lower')
cbar = plt.colorbar()
cbar.set_label('Frequency', size = 18, family = 'DejaVu Serif')
for l in cbar.ax.yaxis.get_ticklabels():
    l.set_family('DejaVu Serif')
    l.set_size(14)
plt.show()

tesla = qrank(R_k['TSLA'])
gme = qrank(R_k['GME'])

plt.scatter(tesla, gme, s = 4)
plt.ylabel('GameStop ranked Returns', **hfont)
plt.xlabel('Tesla ranked Returns', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.autoscale(tight = True)
plt.show()

heatmap, xedges, yedges = np.histogram2d(tesla, gme, bins=(16, 16))

plt.clf()
plt.ylabel('GameStop ranked Returns', **hfont)
plt.xlabel('Tesla ranked Returns', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.imshow(heatmap, origin = 'lower')
cbar = plt.colorbar()
cbar.set_label('Frequency', size = 18, family = 'DejaVu Serif')
for l in cbar.ax.yaxis.get_ticklabels():
    l.set_family('DejaVu Serif')
    l.set_size(14)
plt.show()

In [ ]:
rm = pd.plotting.autocorrelation_plot(R_k['PFE']) 
rm.plot
plt.ylabel('Autocorrelation Pfizer R', **hfont)
plt.xlabel('Lag', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.axis([0, 1500, -0.1, 0.1])
plt.show()

rm2 = pd.plotting.autocorrelation_plot(R_k['PFE'].apply(lambda x: x ** 2)) 
rm2.plot
plt.ylabel('Autocorrelation Pfizer R²', **hfont)
plt.xlabel('Lag', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.axis([0, 1500, -0.2, 0.2])
plt.show()

In [ ]:
pfe1 = R_k['PFE'][:-1]
pfe2 = R_k['PFE'][1:]

plt.scatter(pfe1, pfe2)
plt.ylabel('Pfizer R(t + 1)', **hfont)
plt.xlabel('Pfizer R(t)', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.show()

In [ ]:
heatmap, xedges, yedges = np.histogram2d(qrank(pfe1), qrank(pfe2), bins=(32, 32))

plt.clf()
plt.ylabel('Pfizer ranked R(t + 1)', **hfont)
plt.xlabel('Pfizer ranked R(t)', **hfont)
plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
plt.imshow(heatmap, origin = 'lower')
cbar = plt.colorbar()
cbar.set_label('Frequency', size = 18, family = 'DejaVu Serif')
for l in cbar.ax.yaxis.get_ticklabels():
    l.set_family('DejaVu Serif')
    l.set_size(14)
plt.show()

In [ ]:
locs = ['Duisburg', 'Kiel', 'Lima', 'Houston', 'Anchorage', 'Buenos Aires', 'St. Petersburg', 'Kapstadt', 'Peking', 'Canberra']
stations = [Stations(lat = 51.43255447541353, lon = 6.800788835979209), Stations(lat = 48.112136726828126, lon = 11.577648262186404)]
stations.append(Stations(lat = -23.624908208702756, lon = -46.46244886533699))
stations.append(Stations(lat = 29.761421774856345, lon = -95.34205538962071))
stations.append(Stations(lat = 61.21248946290539, lon = -149.82838837782495))
stations.append(Stations(lat = -34.73232248271822, lon = -58.367538730538925))
stations.append(Stations(lat = 59.9214352225393, lon = 30.353551450337235))
stations.append(Stations(lat = -33.988416855015124, lon = 18.52598649283691))
stations.append(Stations(lat = 39.884160925245155, lon = 116.39399713290861))
stations.append(Stations(lat = -35.32332511410454, lon = 149.1169978599246))

for i, st in enumerate(stations):
    station = st.fetch(1)
    raw_data = Daily(station, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
    raw_data = raw_data.fetch()
    city_data = raw_data['tavg']
    print(len(city_data))
    if i == 9:
        city_data.plot()
        plt.ylabel('Canberra', **hfont)
        plt.xlabel('Date', **hfont)
        plt.yticks(fontname = 'DejaVu Serif', fontsize = 14)
        plt.xticks(fontname = 'DejaVu Serif', fontsize = 14)
        plt.autoscale(axis = 'x', tight = True) ; plt.grid(True)
        plt.show()

In [ ]:
for i, st in enumerate(stations):
    station = st.fetch(1)
    raw_data = Daily(station, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
    raw_data = raw_data.fetch()
    city_data = raw_data['tavg']
    print(locs[i] + ' Mean: ' + str(city_data.mean()))
    print(locs[i] + ' Var: ' + str(city_data.var()))
    print(locs[i] + ' Skew: ' + str(city_data.skew()))
    print(locs[i] + ' Kurtosis: ' + str(city_data.kurtosis() + 3))

In [ ]:
for i, st in enumerate(stations):
    station = st.fetch(1)
    raw_data = Daily(station, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
    raw_data = raw_data.fetch()
    city_data = raw_data['tavg']
    data_n = city_data.sub(city_data.mean()).div(city_data.std())
    data_n.hist(density = True, bins = 100)
    plt.plot(np.linspace(-4, 4, 1000), sps.norm.pdf(np.linspace(-4, 4, 1000)))
    plt.show()
    
    data_n.hist(density = True, bins = 100, log = True)
    plt.semilogy()
    plt.plot(np.linspace(-4, 4, 1000), sps.norm.pdf(np.linspace(-4, 4, 1000)))
    plt.show()

In [ ]:
station1 = stations[0].fetch(1)
raw_data1 = Daily(station1, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
raw_data1 = raw_data1.fetch()
city_data1 = raw_data1['tavg']
data_n1 = city_data1.sub(city_data1.mean()).div(city_data1.std())

station2 = stations[1].fetch(1)
raw_data2 = Daily(station2, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
raw_data2 = raw_data2.fetch()
city_data2 = raw_data2['tavg']
data_n2 = city_data2.sub(city_data2.mean()).div(city_data2.std())

plt.scatter(data_n1, data_n2)
plt.show()

heatmap, xedges, yedges = np.histogram2d(qrank(data_n1), qrank(data_n2), bins=(32, 32))

plt.clf()
plt.ylabel('y')
plt.xlabel('x')
plt.imshow(heatmap, origin = 'lower')
plt.colorbar()
plt.show()

In [ ]:
station1 = stations[0].fetch(1)
raw_data1 = Daily(station1, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
raw_data1 = raw_data1.fetch()
city_data1 = raw_data1['tavg']
data_n1 = city_data1.sub(city_data1.mean()).div(city_data1.std())

station2 = stations[9].fetch(1)
raw_data2 = Daily(station2, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
raw_data2 = raw_data2.fetch()
city_data2 = raw_data2['tavg']
data_n2 = city_data2.sub(city_data2.mean()).div(city_data2.std())

for i in range(len(data_n1) - len(data_n2)):
    c = int(np.random.random() * len(data_n1))
    data_n1 = data_n1.drop(data_n1.index[c])
    
plt.scatter(data_n1, data_n2)
plt.show()

heatmap, xedges, yedges = np.histogram2d(qrank(data_n1), qrank(data_n2), bins=(32, 32))

plt.clf()
plt.ylabel('y')
plt.xlabel('x')
plt.imshow(heatmap, origin = 'lower')
plt.colorbar()
plt.show()

In [ ]:
station1 = stations[5].fetch(1)
raw_data1 = Daily(station1, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
raw_data1 = raw_data1.fetch()
city_data1 = raw_data1['tavg']
data_n1 = city_data1.sub(city_data1.mean()).div(city_data1.std())
d1 = data_n1[:-1]
d2 = data_n1[1:]

print(data_n1)

d3 = pd.plotting.autocorrelation_plot(data_n1) 
d3.plot
#plt.axis([0, 3000, -0.1, 0.1])
plt.show()

plt.scatter(d1, d2)
plt.show()

In [ ]:
station1 = stations[8].fetch(1)
raw_data1 = Daily(station1, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
raw_data1 = raw_data1.fetch()
city_data1 = raw_data1['tavg']
data_dff = city_data1.diff()

data_dff.plot()
plt.show()

In [ ]:
for i, st in enumerate(stations):
    station1 = st.fetch(1)
    raw_data1 = Daily(station1, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
    raw_data1 = raw_data1.fetch()
    city_data1 = raw_data1['tavg']
    data_dff = city_data1.diff()

    print(locs[i] + ' Mean: ' + str(data_dff.mean()))
    print(locs[i] + ' Var: ' + str(data_dff.var()))
    print(locs[i] + ' Skew: ' + str(data_dff.skew()))
    print(locs[i] + ' Kurtosis: ' + str(data_dff.kurtosis() + 3))

In [ ]:
station1 = stations[0].fetch(1)
raw_data1 = Daily(station1, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
raw_data1 = raw_data1.fetch()
city_data1 = raw_data1['tavg']
city_data = city_data1.diff()
data_n1 = city_data.sub(city_data.mean()).div(city_data.std())
print(len(data_n1))

station2 = stations[1].fetch(1)
raw_data2 = Daily(station2, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
raw_data2 = raw_data2.fetch()
city_data2 = raw_data2['tavg']
city_data_ = city_data2.diff()
data_n2 = city_data_.sub(city_data_.mean()).div(city_data_.std())
print(len(data_n2))

plt.scatter(data_n1, data_n2)
plt.show()

heatmap, xedges, yedges = np.histogram2d(qrank(data_n1), qrank(data_n2), bins=(32, 32))

plt.clf()
plt.ylabel('y')
plt.xlabel('x')
plt.imshow(heatmap, origin = 'lower')
plt.colorbar()
plt.show()

In [ ]:
for i, st in enumerate(stations):
    station = st.fetch(1)
    raw_data = Daily(station, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
    raw_data = raw_data.fetch()
    city_data = raw_data['tavg']
    city_data = city_data.diff()
    data_n = city_data.sub(city_data.mean()).div(city_data.std())
    data_n.hist(density = True, bins = 100)
    plt.plot(np.linspace(-4, 4, 1000), sps.norm.pdf(np.linspace(-4, 4, 1000)))
    plt.show()
    
    data_n.hist(density = True, bins = 100, log = True)
    plt.semilogy()
    plt.plot(np.linspace(-4, 4, 1000), sps.norm.pdf(np.linspace(-4, 4, 1000)))
    plt.show()

In [ ]:
station1 = stations[0].fetch(1)
raw_data1 = Daily(station1, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
raw_data1 = raw_data1.fetch()
city_data1 = raw_data1['tavg']
city_data = city_data1.diff()
data_n1 = city_data.sub(city_data.mean()).div(city_data.std())
print(len(data_n1))

station2 = stations[9].fetch(1)
raw_data2 = Daily(station2, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
raw_data2 = raw_data2.fetch()
city_data2 = raw_data2['tavg']
city_data_ = city_data2.diff()
data_n2 = city_data_.sub(city_data_.mean()).div(city_data_.std())
print(len(data_n2))

for i in range(len(data_n1) - len(data_n2)):
    c = int(np.random.random() * len(data_n1))
    data_n1 = data_n1.drop(data_n1.index[c])

plt.scatter(data_n1, data_n2)
plt.show()

heatmap, xedges, yedges = np.histogram2d(qrank(data_n1), qrank(data_n2), bins=(32, 32))

plt.clf()
plt.ylabel('y')
plt.xlabel('x')
plt.imshow(heatmap, origin = 'lower')
plt.colorbar()
plt.show()

In [ ]:
station1 = stations[2].fetch(1)
raw_data1 = Daily(station1, start = datetime(2011, 2, 1), end = datetime(2021, 2, 1))
raw_data1 = raw_data1.fetch()
city_data1 = raw_data1['tavg']
city_data = city_data1.diff()
data_n1 = city_data.sub(city_data.mean()).div(city_data.std())

d1 = data_n1[:-1]
d2 = data_n1[1:]

plt.scatter(d1, d2)
plt.show()

d34 = pd.plotting.autocorrelation_plot(data_n1) 
d34.plot
#plt.axis([0, 3000, -0.1, 0.1])
plt.show()